In [80]:
import numpy as np

import getfem as gf

In [81]:
kappa = 4.         # Thermal conductivity (W/(cm K))
epsilon = 1.   # Plate thickness
Ta = 500.      # Left bound temperature
Tb = 20.       # Right bound temperature

In [82]:
h = 2.                    # Approximate mesh size
elements_degree = 1

In [83]:
# Mesher
mo1 = gf.MesherObject('rectangle', [0., 0.], [100., 25.])
mo2 = gf.MesherObject('ball', [25., 12.5], 8.)
mo3 = gf.MesherObject('ball', [50., 12.5], 8.)
mo4 = gf.MesherObject('ball', [75., 12.5], 8.)
mo5 = gf.MesherObject('union', mo2, mo3, mo4)
mo  = gf.MesherObject('set minus', mo1, mo5)

print('Mesh generation')
gf.util('trace level', 2)   # No trace for mesh generation
mesh = gf.Mesh('generate', mo, h, 2)

Mesh generation


In [84]:
# Boundary Selection
fb1 = mesh.outer_faces_in_box([1., 1.], [99., 24.])    # Boundary of the holes
fb2 = mesh.outer_faces_with_direction([ 1., 0.], 0.01) # Right boundary
fb3 = mesh.outer_faces_with_direction([-1., 0.], 0.01) # Left boundary
fb4 = mesh.outer_faces_with_direction([0.,  1.], 0.01) # Top boundary
fb5 = mesh.outer_faces_with_direction([0., -1.], 0.01) # Bottom boundary
fb6 = mesh.outer_faces_in_ball([25., 12.5], 8.+0.01*h) # Left hole boundary
fb7 = mesh.outer_faces_in_ball([50., 12.5], 8.+0.01*h) # Center hole boundary
fb8 = mesh.outer_faces_in_ball([75., 12.5], 8.+0.01*h) # Right hole boundary

RIGHT_BOUND=1; LEFT_BOUND=2; TOP_BOUND=3; BOTTOM_BOUND=4; HOLE_BOUND=5;
HOLE1_BOUND = 6; HOLE2_BOUND = 7; HOLE3_BOUND = 8;

mesh.set_region( RIGHT_BOUND, fb2)
mesh.set_region(  LEFT_BOUND, fb3)
mesh.set_region(   TOP_BOUND, fb4)
mesh.set_region(BOTTOM_BOUND, fb5)
mesh.set_region(  HOLE_BOUND, fb1)
mesh.set_region( HOLE1_BOUND, fb6)
mesh.set_region( HOLE2_BOUND, fb7)
mesh.set_region( HOLE3_BOUND, fb8)
mesh.region_subtract( RIGHT_BOUND, HOLE_BOUND)
mesh.region_subtract(  LEFT_BOUND, HOLE_BOUND)
mesh.region_subtract(   TOP_BOUND, HOLE_BOUND)
mesh.region_subtract(BOTTOM_BOUND, HOLE_BOUND)

mesh.region_merge(HOLE1_BOUND, HOLE2_BOUND)
mesh.region_merge(HOLE1_BOUND, HOLE3_BOUND)

np.testing.assert_array_equal(mesh.region(HOLE_BOUND), mesh.region(HOLE1_BOUND))

In [85]:
# Link mesh to 1-D FEM
mf = gf.MeshFem(mesh, 1)
# Apply FEM to mesh
mf.set_classical_fem(elements_degree)
# Intergration Method
mim = gf.MeshIm(mesh, elements_degree * 2)

In [87]:
md=gf.Model('real');
# Solving theta(temperature)
md.add_fem_variable('theta', mf)

# 
md.add_initialized_fem_data('Ta', mf, [Ta])
md.add_initialized_fem_data('Tb', mf, [Tb])

md.add_initialized_data('kaeps', [kappa * epsilon])

# Equal to -div(eps*k*grad(theta))
md.add_generic_elliptic_brick(mim, 'theta', 'kaeps')

md.add_Dirichlet_condition_with_simplification('Ta', LEFT_BOUND)
md.add_Dirichlet_condition_with_simplification('Tb', RIGHT_BOUND)





logic_error exception caught


RuntimeError: (Getfem::InterfaceError) -- Error in getfem_models.cc, line 879 : 
Data of null size are not allowed

In [72]:
md.solve()

Trace 2 in getfem_models.cc, line 3463: Generic elliptic: generic matrix assembly
logic_error exception caught


RuntimeError: (Getfem::InterfaceError) -- Error in getfem/getfem_models.h, line 233 : 
This variable is not linked to a fem

In [68]:
md.variable('theta')

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [11]:
import pyvista as pv
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1280, 1024))
display.start()
p = pv.Plotter()
m = pv.read("fem.vtk")
p.add_mesh(m, line_width=5)
pts = m.points
p.add_point_labels(pts, pts[:, 0].tolist(), point_size=10, font_size=10)
p.show(window_size=[512, 384], jupyter_backend='panel', cpos="xy")
display.stop()